# BLEURT computation for the train dataset

## Set-up

In [ ]:
DATA_DIR = "data"

In [ ]:
try:
    import google.colab
    from google.colab import drive
    drive.mount('/content/drive')
    FULL_DATA_DIR = f'/content/drive/My Drive/mbr-reranking/{DATA_DIR}'

    IN_COLAB = True
except:
    FULL_DATA_DIR = DATA_DIR

    IN_COLAB = False

In [ ]:
!pip install evaluate
!pip install git+https://github.com/google-research/bleurt.git

In [ ]:
import evaluate
bleurt = evaluate.load('bleurt')

## Dataset

In [ ]:
reference_sentences = []
generated_sentences = []

with open(f"{FULL_DATA_DIR}/train.eng", 'r') as fp:
    for line in fp:
        reference_sentences.append(line.strip())

with open(f"{FULL_DATA_DIR}/sampled/train.eng", 'r') as fp:
    for line in fp:
        generated_sentences.append(line.strip())

import json

with open(f"{FULL_DATA_DIR}/sampled/train.info.json", 'r') as f:
    metadata = json.load(f)

    samples_per_sentence = metadata["samples_per_sentence"]

    num_different_sentences = len(reference_sentences) // samples_per_sentence

    reference_sentences = [reference_sentences[i // samples_per_sentence] for i in range(len(generated_sentences))]

## Computing BLEURT

In [ ]:
%%time
scores = bleurt.compute(predictions=generated_sentences, references=reference_sentences)
import numpy as np
with open(f"{FULL_DATA_DIR}/sampled/train.scores", 'wb') as f:
    np.save(f, np.array(scores["scores"]))

## End

If in Google Colab, kill the session:

In [ ]:
if IN_COLAB:
    import time
    time.sleep(15)

    from google.colab import runtime
    runtime.unassign()